# Bereiten wir das Script vor

In [1]:
#Imports
import os #Files in listen abgreifen
#import tqdm #anzeigen, wo wir stehen
import textract
import pandas as pd


## Vorbereitung

In [ ]:
#Als Vorbereitung, machen aus dem Inhalt des Folders eine Liste. Es handelt sich hierbei
#um eine Liste aller pdfs im Ordner pdfs. Es sind die Swissmedic Recalls (#ImplantFiles)
pdfs = os.listdir('pdfs')

## Fortschritt anzeigen

In [ ]:
from tqdm import tqdm
for i in tqdm(range(1000)): #100000000000
    elem = i

## Ein File auslesen

In [ ]:
text = textract.process("pdfs/"+pdfs[1]) #method='pdfminer', encoding='ISO-8859-1'

## Alle Files auslesen

In [ ]:
lst = []
mistakes = []
for pdf in tqdm(pdfs):
    try:
        text = textract.process("pdfs/"+pdf, method='pdfminer', encoding='ISO-8859-1')
        lst.append(text)
    except:
        mistakes.append(pdf)

## Bilder auslesen

In [ ]:
text = textract.process("pdfs/"+pdfs[0], method='tesseract') #Nicht ideal, aber gut

## Alles zusammen

In [ ]:
#Rohe Analyse
for pdf in tqdm(pdfs):
    try:
        text = textract.process("pdfs/"+pdf, method='pdfminer', encoding='ISO-8859-1')
        text = text.decode("ISO-8859-1").replace("\n", " ")
        if len(text) < 40:
            text = textract.process("pdfs/"+pdf, method='tesseract', language='deu')
            text = text.decode("ISO-8859-1").replace("\n", " ")
            
        mini_dict = {'Text':text,
                     'File':pdf}
        lst.append(mini_dict)
    except:
        mini_dict = {'Text':'Fehlermeldung',
                     'File':pdf}
        lst.append(mini_dict)

In [ ]:
#Erste Säuberung
df = pd.DataFrame(lst)
def date(elem):
    elem = elem[3:11]
    return elem
df['date'] = df['File'].apply(date)
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')
df.index = df['date']
#Suchen wir noch nach Implantaten
df['implant'] = df['Text'].str.contains('implant')
df.to_csv('datafile.csv')